# 5-5 Adjust imbalanced data

## Under sampling

### Pandas

In [1]:
import pandas as pd

In [2]:
pd_reservation = pd.read_parquet("../../data/reservation.parquet")

In [3]:
pd_reservation.head()

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT


In [4]:
# under sampling

# split data
majority = pd_reservation.query("status == 'reserved'")

minority = pd_reservation.query("status == 'canceled'")


# concatenation
pd.concat([
    minority,

    # under sample
    majority.sample(len(minority))
])

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
11,12,1649,457529,2014-01-02 00:57:10+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,4,145200,canceled,2014-05-12 02:45:20+09:00
17,18,2681,158800,2014-01-03 07:24:45+09:00,2015-01-03 00:00:00+09:00,2015-01-04 00:00:00+09:00,1,4,50400,canceled,2014-10-30 13:45:51+09:00
27,28,4843,130891,2014-01-04 16:21:42+09:00,2015-01-02 00:00:00+09:00,2015-01-03 00:00:00+09:00,1,1,5300,canceled,2014-05-23 09:25:18+09:00
33,34,301,434094,2014-01-05 12:22:22+09:00,2015-01-01 00:00:00+09:00,2015-01-02 00:00:00+09:00,1,5,70000,canceled,2014-07-05 14:00:19+09:00
36,37,2715,307335,2014-01-05 15:31:53+09:00,2015-01-02 00:00:00+09:00,2015-01-03 00:00:00+09:00,1,3,45600,canceled,2014-01-22 20:08:21+09:00
...,...,...,...,...,...,...,...,...,...,...,...
508981,508982,2415,119850,2015-10-13 21:50:04+09:00,2015-11-17 00:00:00+09:00,2015-11-18 00:00:00+09:00,1,1,10900,reserved,NaT
1825076,1825077,3546,31613,2019-01-27 23:59:53+09:00,2019-03-24 00:00:00+09:00,2019-03-25 00:00:00+09:00,1,2,30400,reserved,NaT
1287,1288,1957,40180,2014-02-06 11:00:28+09:00,2015-01-04 00:00:00+09:00,2015-01-05 00:00:00+09:00,1,2,30000,reserved,NaT
1249493,1249494,3060,75368,2017-08-20 09:48:40+09:00,2018-06-16 00:00:00+09:00,2018-06-17 00:00:00+09:00,1,3,34200,reserved,NaT


In [5]:
# by `imbalanced-learn`
from imblearn.under_sampling import RandomUnderSampler

# split features and target
feature_cols = ["length_of_stay", "people_num", "total_price"]
target_col   = "status"

X = pd_reservation[feature_cols]
y = pd_reservation[target_col]

# Create object executes under sample
sampler = RandomUnderSampler()

# Execute under sampling by passing X and y as arguments
X_sample, y_sample = sampler.fit_resample(X, y)

In [6]:
X_sample

,length_of_stay,people_num,total_price
11,3,4,145200
17,1,4,50400
27,1,1,5300
33,1,5,70000
36,1,3,45600
...,...,...,...
1657173,1,1,4300
1956563,1,1,6700
58485,1,2,24800
767010,2,3,49800


### Polars

In [7]:
import polars as pl

In [8]:
pl_reservation = pl.read_parquet("../../data/reservation.parquet")

In [9]:
pl_reservation.head()

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null


In [10]:
majority = pl_reservation.filter(pl.col("status") == "reserved")
minority = pl_reservation.filter(pl.col("status") == "canceled")

pl.concat([
    minority,
    majority.sample(len(minority))
])

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
12,1649,457529,2014-01-02 00:57:10 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,4,145200,"""canceled""",2014-05-12 02:45:20 JST
18,2681,158800,2014-01-03 07:24:45 JST,2015-01-03 00:00:00 JST,2015-01-04 00:00:00 JST,1,4,50400,"""canceled""",2014-10-30 13:45:51 JST
28,4843,130891,2014-01-04 16:21:42 JST,2015-01-02 00:00:00 JST,2015-01-03 00:00:00 JST,1,1,5300,"""canceled""",2014-05-23 09:25:18 JST
34,301,434094,2014-01-05 12:22:22 JST,2015-01-01 00:00:00 JST,2015-01-02 00:00:00 JST,1,5,70000,"""canceled""",2014-07-05 14:00:19 JST
37,2715,307335,2014-01-05 15:31:53 JST,2015-01-02 00:00:00 JST,2015-01-03 00:00:00 JST,1,3,45600,"""canceled""",2014-01-22 20:08:21 JST
…,…,…,…,…,…,…,…,…,…,…
1710332,2332,64082,2018-10-14 21:25:05 JST,2019-03-17 00:00:00 JST,2019-03-18 00:00:00 JST,1,5,28500,"""reserved""",null
297635,3812,442138,2015-04-03 14:08:59 JST,2015-12-02 00:00:00 JST,2015-12-04 00:00:00 JST,2,5,55000,"""reserved""",null
1032047,2433,15644,2017-02-03 02:10:23 JST,2017-12-19 00:00:00 JST,2017-12-20 00:00:00 JST,1,2,51400,"""reserved""",null
